In [70]:
''' 
資料mapping至google drive
把資料寫在/content/drive
即可保存在google drive內
'''
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
!pip install line-bot-sdk flask-ngrok

In [72]:
from flask import Flask, request, abort, jsonify
import json
from flask_ngrok import run_with_ngrok

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)

In [73]:
app = Flask(__name__,static_url_path = "/material", static_folder = "./material/")
run_with_ngrok(app)

line_bot_api = LineBotApi("9K2lbiSmQIY+xnkbt0MqxyD2SLrHvLY8nTb9Au76oyM+fG7ua4k6khpTN1TMM2aHNpQPDCAgSTgeT/hKZyS7SycCqx+OJaKWezw/tGlIk8P2rOtNIfgR9QEwmPokzcVmn4U1UW802Gs76jLK0cWShAdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("4257df85eafe8d3c8de0ca1cd8db08a9")


In [74]:
@app.route("/", methods=['POST'])
def callback():
  signature = request.headers['X-Line-Signature']
  body = request.get_data(as_text=True)
  print(body)
  f = open("/content/drive/MyDrive/ai-event.log","a")
  f.write(body)
  f.close()

  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

In [75]:
'''消息素材準備'''

#將消息模型，文字收取消息與文字寄發消息引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,ImageSendMessage
)

specific_text_message=TextSendMessage("官方建議！用戶以文字消息觸發特殊功能的時候，要加上特殊符號")

analyze_truth_image_message=ImageSendMessage(
      original_content_url='https://ibw.bwnet.com.tw/ac_gallery/2019/10/31095c35-c141-b1a9-e881-db41f467a91b_620.jpg',
      preview_image_url='https://imgcdn.cna.com.tw/www/WebPhotos/1024/20210127/1024x768_20210127000006.jpg'
      )


In [76]:
'''
設計一個字典
當用戶輸入相應文字消息時，系統會從此挑揀消息
'''

#根據自訂義菜單四張故事線的圖，設定相對應image
template_message_dict = {
    "#1":specific_text_message,
    "#2":analyze_truth_image_message
}

In [77]:
#用戶發文字消息時，按條件內容，回傳文字消息
@handler.add(MessageEvent,message=TextMessage)
def handle_message(event):

  if(event.message.text.find('#')!= -1):
    line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
    )
  else:
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="你輸入無效，請輸入#1或#2")
    )


In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2c5921a6b3de.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U84891643b3f7d23f39ca3989530192c5","events":[{"type":"message","message":{"type":"text","id":"14108095072594","text":"1"},"timestamp":1621881089236,"source":{"type":"user","userId":"U2f25cfa41ca384946f24585d45019074"},"replyToken":"935bc97e47e64a408986a0877305b54b","mode":"active"}]}


127.0.0.1 - - [24/May/2021 18:31:31] "POST / HTTP/1.1" 200 -


{"destination":"U84891643b3f7d23f39ca3989530192c5","events":[{"type":"message","message":{"type":"text","id":"14108095317985","text":"#1"},"timestamp":1621881095120,"source":{"type":"user","userId":"U2f25cfa41ca384946f24585d45019074"},"replyToken":"65377979c5644ef6b65ab8d93498929e","mode":"active"}]}


127.0.0.1 - - [24/May/2021 18:31:36] "POST / HTTP/1.1" 200 -


{"destination":"U84891643b3f7d23f39ca3989530192c5","events":[{"type":"message","message":{"type":"text","id":"14108095487633","text":"#2"},"timestamp":1621881099332,"source":{"type":"user","userId":"U2f25cfa41ca384946f24585d45019074"},"replyToken":"9deb3eb2363545318f4661e7d6f02a3a","mode":"active"}]}


127.0.0.1 - - [24/May/2021 18:31:40] "POST / HTTP/1.1" 200 -
